# Climatology Functions

```{warning} This is not meant to be a standalone notebook.
This notebook is part of the process we have for adding entries to the NCL Index and is not meant to be used as tutorial or example code.
```

## Functions covered

- [calcDayAnomTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/calcDayAnomTLL.shtml)
- [calcMonAnomTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/calcMonAnomTLL.shtml)
- [clmDayTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/clmDayTLL.shtml)
- [clmMonTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/clmMonTLL.shtml)
- [month_to_season](https://www.ncl.ucar.edu/Document/Functions/Contributed/month_to_season.shtml)
- [rmMonAnnCycTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/rmMonAnnCycTLL.shtml)
- [stdMonTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/stdMonTLL.shtml)

## NCL code

```{literalinclude} ../ncl_raw/climo.ncl

```

## Python Functionality

In [11]:
import math
import xarray as xr
import geocat.comp as gc

results = {}

ds_m = xr.open_dataset(
    "../ncl_raw/b.e21.BHIST.f09_g17.CMIP6-historical.003.cice.h.aice.185001-201412.nc"
)
ds_d = xr.open_dataset(
    "../ncl_raw/b.e21.BHIST.f09_g17.CMIP6-historical.003.cice.h1.aice_d.18500101-20141231.nc"
)

# calcDayAnomTLL
aice = ds_d.aice_d[54749:58399, 365, 300]
DayTLL = aice.groupby(aice.time.dt.dayofyear)
clmDayTLL = DayTLL.mean(dim="time")
calcDayAnomTLL = DayTLL - clmDayTLL
results["calcDayAnomTLL"] = calcDayAnomTLL.item(0)

# calcMonAnomTLL
aice = ds_m.aice[1799:1919, 365, 300]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
calcMonAnomTLL = MonTLL - clmMonTLL
results["calcMonAnomTLL"] = calcMonAnomTLL.item(0)

# clmDayTLL
aice = ds_d.aice_d[54749:58399, 365, 300]
DayTLL = aice.groupby(aice.time.dt.dayofyear)
clmDayTLL = DayTLL.mean(dim="time")
results["clmDayTLL"] = clmDayTLL.item(0)

# clmMonTLL
aice = ds_m.aice[1799:1919, 365, 300]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
results["clmMonTLL"] = clmMonTLL.item(0)

# month_to_season
aice = ds_m.aice[1799:1919, 365, 300]
mon_to_season = gc.climatologies.month_to_season(aice, "ASO")
results["month_to_season"] = mon_to_season.item(0)

# rmMonAnnCycTLL
aice = ds_m.aice[1799:1919, 365, 300]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
rmMonAnnCycTLL = MonTLL - clmMonTLL
results["rmMonAnnCycTLL"] = rmMonAnnCycTLL.item(0)

# stdMonTLL
aice = ds_m.aice[1799:1919, 365, 300]
MonTLL = aice.groupby(aice.time.dt.month)
stdMonTLL = MonTLL.std(ddof=1)
results["stdMonTLL"] = stdMonTLL.item(0)

## Comparison

In [18]:
# dictionary of climatology functions results
ncl_results = {
    "calcDayAnomTLL": 0.2156208,
    "calcMonAnomTLL": 0.09780759,
    "clmDayTLL": 0.4031426,
    "clmMonTLL": 0.1261306,
    "month_to_season": 4.922778e-06,
    "rmMonAnnCycTLL": 0.09780759,
    "stdMonTLL": 0.1073168,
}

for c in ncl_results.keys() & results.keys():
    print(f"{c}: \n\tpython:\t\t{results[c]}\n\tncl:\t\t{ncl_results[c]}\n")
    assert math.isclose(
        ncl_results[c], results[c], rel_tol=1e-06
    )  # within 6 decimal points

rmMonAnnCycTLL: 
	python:		0.09780758619308472
	ncl:		0.09780759

calcMonAnomTLL: 
	python:		0.09780758619308472
	ncl:		0.09780759

clmDayTLL: 
	python:		0.4031425416469574
	ncl:		0.4031426

clmMonTLL: 
	python:		0.126130610704422
	ncl:		0.1261306

stdMonTLL: 
	python:		0.10731684416532516
	ncl:		0.1073168

month_to_season: 
	python:		4.9227783165406436e-06
	ncl:		4.922778e-06

calcDayAnomTLL: 
	python:		0.21562078595161438
	ncl:		0.2156208



## Differences

In [14]:
for c in ncl_results.keys() & results.keys():
    print(f"{c}:")
    print(f"\t{results[c] - ncl_results[c]}")

rmMonAnnCycTLL:
	-3.806915283011136e-09
calcMonAnomTLL:
	-3.806915283011136e-09
clmDayTLL:
	-5.835304262014063e-08
clmMonTLL:
	1.0704421987695056e-08
stdMonTLL:
	4.416532516093863e-08
month_to_season:
	3.165406432182791e-13
calcDayAnomTLL:
	-1.404838562146793e-08
